# Paper plots

This notebook generates the plots used in the paper.  

**This notebook depends on:**
* Running `./summarize_human_model_behavior.ipynb` (PUBLIC USE)*

## setup

#### Load packages

In [ ]:
import os
import sys
import urllib, io

sys.path.append('./analysis_helpers')
from importlib import reload

import numpy as np
import scipy.stats as stats
import pandas as pd

import analysis_helpers as h

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

from tqdm.notebook import tqdm

import  matplotlib
from matplotlib import pylab, mlab, pyplot
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
plt.style.use('seaborn-white')

import seaborn as sns

%matplotlib inline
import scipy.stats
import sklearn.metrics
import random

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

#### options

In [ ]:
# display all columns
pd.set_option('display.max_columns', None)

# seaborn plotting themes
sns.set_context('talk')
sns.set_style("whitegrid")

#### set up paths and directories

In [ ]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
## add helpers to python path
if os.path.join(analysis_dir,'utils') not in sys.path:
    sys.path.append(os.path.join(analysis_dir,'utils'))   

def make_dir_if_not_exists(dir_name):   
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

## create directories that don't already exist        
result = [make_dir_if_not_exists(x) for x in [results_dir,plot_dir,csv_dir]]

## load data

In [ ]:
AMH = pd.read_csv(os.path.join(csv_dir,"summary/model_human_accuracies.csv"))
AMCK = pd.read_csv(os.path.join(csv_dir,"summary/model_human_CohensK.csv"))
AMRMSE = pd.read_csv(os.path.join(csv_dir,"summary/model_human_pearsonsr_rmse.csv"))
AH = pd.read_csv(os.path.join(csv_dir,"summary/human_accuracy_by_scenario.csv"))
ACK = pd.read_csv(os.path.join(csv_dir,"summary/human_pairwiseCohensKs_by_scenario.csv"))
AR = pd.read_csv(os.path.join(csv_dir,"summary/human_pairwiseCorrs_by_scenario.csv"))

In [ ]:
# # get percentiles for AMRMSE
# AMRMSE = AMRMSE.groupby(['ModelID','scenario']).agg({
#     'Model Kind':'first',
#     'RMSE':['mean',lambda x:np.percentile(x,2.5),lambda x:np.percentile(x,97.5)]
# })

# # get rid of multicols
# AMRMSE.columns = [''.join(col) if type(col) is tuple else col for col in AMRMSE.columns.values]

# AMRMSE.reset_index(inplace=True)

# AMRMSE = AMRMSE.rename(columns={
#     "RMSEmean":"RMSE_mean",
#     "RMSE<lambda_0>":"RMSE_lb",
#     "RMSE<lambda_1>":"RMSE_ub",
#     "Model Kindfirst":"Model Kind"
# })

In [ ]:
# merge dataframes
_AMCK = AMCK[["ModelID","scenario","Cohens_k_lb","Cohens_k_med","Cohens_k_ub"]]
# _AMRMSE = AMRMSE[["ModelID","scenario","RMSE_mean","RMSE_lb","RMSE_ub",]]
_AMRMSE = AMRMSE[["ModelID","scenario","RMSE","pearsons_r"]]
MH = pd.merge(AMH,_AMCK,on=['ModelID',"scenario"])
MH = pd.merge(MH,_AMRMSE,on=['ModelID',"scenario"])

In [ ]:
SCENARIOS = sorted(MH['scenario'].unique())

In [ ]:
#kick out SVG/RPIN additional seeds
MH = MH[~(((MH['Model'] == 'SVG') | (MH['Model'] == 'RPIN')) & (MH['Encoder Training Seed'] != 0.0))]

In [ ]:
#choose readout and training type
MH_both = MH[(MH['Readout Type'] == "B") & (MH['Dynamics Training Dataset Type'] == "all")]

In [ ]:
MH_readouts = MH[(MH['Dynamics Training Dataset Type'] == "all")]

In [ ]:
MH_trainings = MH[(MH['Readout Type'] == "B")].copy()

In [ ]:
#collapse model names HACK
MH_trainings['Model Kind'] = MH_trainings['Model Kind'].apply(lambda x: x.replace("same_","all_"))
MH_trainings['Model Kind'] = MH_trainings['Model Kind'].apply(lambda x: x.replace("all_but_this","all"))

## Model/human comparision plot

In [ ]:
#set up model names
MODELS = dict(MH_both.groupby('Model Kind')['Model'].first())

In [ ]:
assert np.all(MH_both.groupby('Model Kind')['ModelID'].count() == 8)

In [ ]:
#rename these by hand if necessary
MODELS = {
 'SVG_VGG_0.0_VAE_all_VAE_0_same': 'SVG',
 'OP3_OP3 encoder_0.0_Image Reconstruction_all_Image Reconstruction_0_same': 'OP3',
 'CSWM_CSWM encoder_0.0_Contrastive_all_Contrastive_0_same': 'CSWM',
 'RPIN_R-CNN_0.0_L2 on 2D position_all_L2 on 2D position_0_same': 'RPIN',
 'VGGFrozenMLP_VGG_nan_nan_nan_L2 on latent_0_same': 'pVGG-mlp',
 'VGGFrozenLSTM_VGG_nan_nan_nan_L2 on latent_0_same': 'pVGG-lstm',
 'DEITFrozenMLP_DEIT_nan_nan_nan_L2 on latent_0_same': 'pDEIT-mlp',
 'DEITFrozenLSTM_DEIT_nan_nan_nan_L2 on latent_0_same': 'pDEIT-lstm',
 'GNS_nan_nan_nan_nan_L2 on particle 3D positions_1_same': 'GNS',
 'GNS-ransac_nan_nan_nan_nan_L2 on particle 3D positions_1_same': 'GNS-R',
 'DPI_nan_nan_nan_nan_L2 on particle 3D positions_1_same': 'DPI',
}

In [ ]:
COLORS = {
    'SVG':'orangered',
    'OP3':'darkorange',
    'CSWM':'gold',
    'RPIN':'greenyellow',
    'pDEIT-lstm':'royalblue',
    'pDEIT-mlp':'blue',
    'pVGG-lstm':'aquamarine',
    'pVGG-mlp':'turquoise',
    'GNS':'slateblue',
    'GNS-R':'orchid',
    'DPI':'hotpink',
}

### Pretty plotting things

In [ ]:
import matplotlib as mpl
from svgpathtools import svg2paths
from svgpath2mpl import parse_path
def marker_from_svg(path):
    path, attributes = svg2paths(path)
    marker = parse_path(attributes[0]['d'])
#     marker = parse_path(path)
    marker.vertices -= marker.vertices.mean(axis=0)
    marker = marker.transformed(mpl.transforms.Affine2D().rotate_deg(180))
    marker = marker.transformed(mpl.transforms.Affine2D().scale(-1,1))
    return marker

In [ ]:
ICONS = {
     'clothiness':"<",
     'collision':">",
     'containment':"X",
     'dominoes':"p",
     'drop':"d",
     'linking':"h",
     'rollingsliding':"v",
     'towers':"P"
}

In [ ]:
svg_dir = os.path.join(proj_dir,"figures/icons/SVG")

ICONS = {
     'clothiness':marker_from_svg(os.path.join(svg_dir,"clothiness.svg")),
     'collision':marker_from_svg(os.path.join(svg_dir,"collision.svg")),
     'containment':marker_from_svg(os.path.join(svg_dir,"containment.svg")),
     'dominoes':marker_from_svg(os.path.join(svg_dir,"dominoes.svg")),
     'drop':marker_from_svg(os.path.join(svg_dir,"drop.svg")),
     'linking':marker_from_svg(os.path.join(svg_dir,"linking.svg")),
     'rollingsliding':marker_from_svg(os.path.join(svg_dir,"rollingsliding.svg")),
     'towers':marker_from_svg(os.path.join(svg_dir,"towers.svg"))
}

In [ ]:
#set up color map over models
from matplotlib import cm

cmap = cm.get_cmap('tab20_r')
# cmap = cmap(np.linspace(0, 1, 22))
cmap = cmap.colors

In [ ]:
MARKER_SIZE = 20
MEAN_MULTIPLIER = 0.8
ALPHA = 0.2
SCEN_SHIFT = .06
N_ITER = 1000
FIGSIZE = (7,7)

In [ ]:
def bootstrap_means(means):
    """Bootstraps over the means across scenarios to generate lower and upper confidence bounds"""
    bootstrapped_means = [np.mean(np.random.choice(means,size=len(AH),replace=True)) for _ in range(N_ITER)]
    lb = np.percentile(bootstrapped_means,2.5)
    ub = np.percentile(bootstrapped_means,97.5)
    mean = np.mean(means)
    boot_mean = np.mean(bootstrapped_means)
    boot_median = np.percentile(bootstrapped_means,50)
    return mean,lb,ub,boot_mean,boot_median

### Accuracy

In [ ]:
fig, axis = plt.subplots(figsize=FIGSIZE)

# human zone
## human
mean_human,lb_human,ub_human,_,_ = bootstrap_means(AH['obs_mean'])
axis.axhspan(lb_human,ub_human,
              color="black",
               alpha=0.1)

for si,scenario in enumerate(SCENARIOS):
    #plot human data
    axis.plot(0+si*.03,AH[AH['scenario']==scenario]['obs_mean'],
              label="Human",
              marker=ICONS[scenario],
              fillstyle='full',
#               markerfacecolor="black",
              color="black",
              alpha=.3, 
              ms=MARKER_SIZE)
    #plot model data
    for x,(model,model_name) in enumerate(MODELS.items()):
        axis.plot(x+1+si*SCEN_SHIFT,MH_both[(MH_both['scenario']==scenario) & (MH_both['Model Kind'] == model)]['model_correct'],
                  marker=ICONS[scenario],
                  fillstyle='full',
                  markerfacecolor=COLORS[model_name],
                  color="black",
                  label=scenario,
                  alpha=.3,
                  ms=MARKER_SIZE)
#plot means
## human
lb_human = abs(mean_human - lb_human) #as we error relative to mean
ub_human = abs(mean_human - ub_human)
axis.errorbar(0-.2,mean_human,
              yerr=([lb_human],[ub_human]),
              label="Human",
              color="black",
              marker = "o",
              ms=MARKER_SIZE*MEAN_MULTIPLIER)
## models
for x,(model,model_name) in enumerate(MODELS.items()):
    mean,lb,ub,_,_ = bootstrap_means(MH_both[(MH_both['Model Kind'] == model)]['model_correct'])
    axis.errorbar(x+1-.2,mean,([abs(mean-lb)],[abs(mean-ub)]),
      marker="o",
      color=COLORS[model_name],
      label=model_name,
      ms=MARKER_SIZE*MEAN_MULTIPLIER)

#add labels
axis.set_xticks(np.arange(len(MODELS)+1))
axis.set_xticklabels(["Human"] + list(MODELS.values()),rotation=45,ha='right')
axis.set_title("Accuracy")
axis.set_ylabel("Accuracy")
axis.set_ybound((0.3,1))
plt.savefig(os.path.join(plot_dir,"human_model_accuracy.pdf"),bbox_inches='tight')
plt.show()

### Cohen's $\kappa$

In [ ]:
fig, axis = plt.subplots(figsize=FIGSIZE)

# human zone
## human
mean_human,lb_human,ub_human,_,_ = bootstrap_means(ACK['corr_med'])
axis.axhspan(lb_human,ub_human,
              color="black",
               alpha=0.1)

for si,scenario in enumerate(SCENARIOS):
    #plot human data
    axis.plot(0+si*.03,ACK[ACK['scenario']==scenario]['corr_med'],
              label="Human",
              marker=ICONS[scenario],
              fillstyle='full',
#               markerfacecolor="black",
              color="black",
              alpha=.3, 
              ms=MARKER_SIZE)
    #plot model data
    for x,(model,model_name) in enumerate(MODELS.items()):
        axis.plot(x+1+si*SCEN_SHIFT,MH_both[(MH_both['scenario']==scenario) & (MH_both['Model Kind'] == model)]['Cohens_k_med'],
                  marker=ICONS[scenario],
                  fillstyle='full',
                  markerfacecolor=COLORS[model_name],
                  color="black",
                  label=model_name,
                  alpha=.3,
                  ms=MARKER_SIZE)
#plot means
## human
lb_human = abs(mean_human - lb_human) #as we error relative to mean
ub_human = abs(mean_human - ub_human)
axis.errorbar(0-.2,mean_human,
              yerr=([lb_human],[ub_human]),
              label="Human",
              color="black",
              marker = "o",
              ms=MARKER_SIZE*MEAN_MULTIPLIER)
## models
for x,(model,model_name) in enumerate(MODELS.items()):
    mean,lb,ub,_,_ = bootstrap_means(MH_both[(MH_both['Model Kind'] == model)]['Cohens_k_med'])
    axis.errorbar(x+1-.2,mean,([abs(mean-lb)],[abs(mean-ub)]),
      marker="o",
      color=COLORS[model_name],
      label=model_name,
      ms=MARKER_SIZE*MEAN_MULTIPLIER)
   
#add labels
axis.set_xticks(np.arange(len(MODELS)+1))
axis.set_xticklabels(["Human"] + list(MODELS.values()),rotation=45,ha='right')
axis.set_title("Cohen's $\kappa$")
axis.set_ylabel("$\kappa$ within human/between human & model")
axis.set_ybound((-.25,1))
plt.savefig(os.path.join(plot_dir,"human_model_CohensK.pdf"),bbox_inches='tight')
plt.show()

### Correlation between model & average human

In [ ]:
fig, axis = plt.subplots(figsize=FIGSIZE)

# human zone
lb = AR['r_lb'].mean()
ub = AR['r_ub'].mean()
axis.axhspan(lb,ub,
              color="black",
               alpha=0.1)

for si,scenario in enumerate(SCENARIOS):
    #plot human data
    axis.plot(0+si*.03,AR[AR['scenario']==scenario]['r_mean'],
              label="Human",
              marker=ICONS[scenario],
              fillstyle='full',
#               markerfacecolor="black",
              color="black",
              alpha=.3, 
              ms=MARKER_SIZE)
    #plot model data
    for x,(model,model_name) in enumerate(MODELS.items()):
        axis.plot(x+1+si*SCEN_SHIFT,MH_both[(MH_both['scenario']==scenario) & (MH_both['Model Kind'] == model)]['pearsons_r'],
                  marker=ICONS[scenario],
                  fillstyle='full',
                  markerfacecolor=COLORS[model_name],
                  color="black",
                  label=model_name,
                  alpha=.3,
                  ms=MARKER_SIZE)
# plot means
# human
mean = AR['r_mean'].mean()
lb = abs(mean-AR['r_lb'].mean())
ub = abs(mean-AR['r_ub'].mean())
assert AR['r_ub'].mean() == ub+mean
axis.errorbar(0-.2,mean,
              yerr=([lb],[ub]),
              label="Human",
              color="black",
              marker = "o",
              ms=MARKER_SIZE*MEAN_MULTIPLIER,
)
## models
for x,(model,model_name) in enumerate(MODELS.items()):
    print("error bar",model_name,model)
    pearsons_rs = MH_both[(MH_both['Model Kind'] == model)]['pearsons_r']
    # drop nans
    pearsons_rs = pearsons_rs[~np.isnan(pearsons_rs)]
    mean,lb,ub,_,_ = bootstrap_means(pearsons_rs)
    print(mean,lb,ub)
    axis.errorbar(x+1+-.2,mean,([abs(mean-lb)],[abs(mean-ub)]),
      marker="o",
      color=COLORS[model_name],
      label=model_name,
      ms=MARKER_SIZE*MEAN_MULTIPLIER)
   
#add labels
axis.set_xticks(np.arange(len(MODELS)+1))
axis.set_xticklabels(["Human"] + list(MODELS.values()),rotation=45,ha='right')
axis.set_title("Correlation to average human response")
axis.set_ylabel("Pearson's r")
# axis.set_ybound((0,0.08))
plt.savefig(os.path.join(plot_dir,"human_model_distance.pdf"),bbox_inches='tight')
plt.show()


## Adversarial stimuli

### Model/human correlation in full vs adversarial stim set
Assumes that `summary/model_human_pearsonsr_rmse_ADVERSARIAL.csv` contains a parallel version of the data in `summary/model_human_pearsonsr_rmse.csv`, but only with adversarial stimuli (ie. those that humans get wrong in >50% of cases). Use `summarize_human_model_behavior.ipynb` to generate the .csvs

In [ ]:
# read in adversarial only correlations
AMRMSE = pd.read_csv(os.path.join(csv_dir,"summary/model_human_pearsonsr_rmse.csv"))
AMRMSE_adversarial = pd.read_csv(os.path.join(csv_dir,"summary/model_human_pearsonsr_rmse_ADVERSARIAL.csv"))

In [ ]:
#kick out SVG/RPIN additional seeds
AMRMSE = AMRMSE[~(((AMRMSE['Model'] == 'SVG') | (AMRMSE['Model'] == 'RPIN')) & (AMRMSE['Encoder Training Seed'] != 0.0))]
AMRMSE_adversarial = AMRMSE_adversarial[~(((AMRMSE_adversarial['Model'] == 'SVG') | (AMRMSE_adversarial['Model'] == 'RPIN')) & (AMRMSE_adversarial['Encoder Training Seed'] != 0.0))]

In [ ]:
#choose readout and training type
AMRMSE = AMRMSE[(AMRMSE['Readout Type'] == "B") & (AMRMSE['Dynamics Training Dataset Type'] == "all")]
AMRMSE_adversarial = AMRMSE_adversarial[(AMRMSE_adversarial['Readout Type'] == "B") & (AMRMSE_adversarial['Dynamics Training Dataset Type'] == "all")]

In [ ]:
fig, axis = plt.subplots(figsize=FIGSIZE)

for mi,(model,model_name) in enumerate(MODELS.items()):
    for scenario in SCENARIOS:
        axis.plot(AMRMSE[(AMRMSE['scenario'] == scenario) & (AMRMSE['Model Kind'] == model)]['pearsons_r'].head(1), #head is an ugly hack for duplicates
                  AMRMSE_adversarial[(AMRMSE_adversarial['scenario'] == scenario) & (AMRMSE_adversarial['Model Kind'] == model)]['pearsons_r'],
                  marker=ICONS[scenario],
                  fillstyle='full',
                  markerfacecolor=COLORS[model_name],
                  color="black",
                  label=model_name,
                  alpha=.3,
                  ms=MARKER_SIZE)
        #add colored text label
        fig.text(.95,.15+mi*1/15,model_name,color=COLORS[model_name],alpha=.1,fontsize='small',fontweight='bold')
# plt.legend()
plt.xlabel("Pearson's r on all stimuli")
plt.ylabel("Pearson's r on adversarial stimuli")
plt.title("Pearson's r between model and humans")
plt.show()

### Readout type
only vision models

view + simulation is not better than just view

compare B & C on per model basis
paired lines
average across scenario
col: model
    subcol0 C
    subcol1 B
    subcol2 A
y accuracy
color: model

In [ ]:
# get the models we have more than one readout type for
readout_models = [m for m,c in (MH_readouts.groupby('Model Kind')['Readout Type'].nunique()).items() if c >= 2]

In [ ]:
MODELS_readout = {key:value for key,value in MODELS.items() if key in readout_models}

In [ ]:
READOUTS = ["C","B","A"]

READOUT_MARKER = {
    'A': '^',
    'B': 'o',
    'C': 's'
}

In [ ]:
fig, axis = plt.subplots(figsize=(9,3))

# human zone
## human
mean_human,lb_human,ub_human,_,_ = bootstrap_means(AH['obs_mean'])
axis.axhspan(lb_human,ub_human,
              color="black",
               alpha=0.1)

#plot means
## human
lb_human = abs(mean_human - lb_human) #as we error relative to mean
ub_human = abs(mean_human - ub_human)
axis.errorbar(0-.2,mean_human,
              yerr=([lb_human],[ub_human]),
              label="Human",
              color="black",
              marker = "o",
              ms=MARKER_SIZE*MEAN_MULTIPLIER)
## models
for x,(model,model_name) in enumerate(MODELS_readout.items()):
    means = []
    lbs = []
    ubs = []
    for o,readout in enumerate(READOUTS):
        mean,lb,ub,_,_ = bootstrap_means(MH_readouts[(MH_readouts['Model Kind'] == model)&
                                                     (MH_readouts['Readout Type'] == readout)]['model_correct'])
        lb = abs(mean-lb); ub = abs(mean-ub); 
        means.append(mean); lbs.append(lb), ubs.append(ub)
        axis.errorbar(x+.66+0.33 * o,mean,([lb],[ub]),
          marker=READOUT_MARKER[readout],
          color=COLORS[model_name],
          label=model_name,
          ms=10)
    plt.plot([x+.66+0.33 * o for o in range(len(means))],means,color=COLORS[model_name])

#add labels
axis.set_xticks(np.arange(len(MODELS_readout)+1))
axis.set_xticklabels(["Human"] + list(MODELS_readout.values()),rotation=45,ha='right')
axis.set_title("Accuracy per readout type")
axis.set_ylabel("Accuracy")
axis.set_ybound((0.3,1))
plt.savefig(os.path.join(plot_dir,"human_model_readout_accuracy.pdf"),bbox_inches='tight')
plt.show()

### Training Type
do some models generalize better than others? Are any of the scenarios more important to include
points for scenarios
maybe: same as cohens with interconnected lines
or paired barplot with errorbars

or two plots: connected lines
* averaging across scenarios (are some models better at generalizing?)
* are there scenarios that are important to include?

In [ ]:
# get the models we have more than one readout type for
trainings_models = [m for m,c in (MH_trainings.groupby('Model Kind')['Dynamics Training Dataset Type'].nunique()).items() if c >= 2]

In [ ]:
MODELS_trainings = {key:value for key,value in MODELS.items() if key in trainings_models}

In [ ]:
TRAININGS = ["all","all_but_this","same"]

TRAINING_FILL = {
    'all': 'full',
    'all_but_this': 'left',
    'same': 'none'
}

In [ ]:
fig, axis = plt.subplots(figsize=(9,3))

# human zone
## human
mean_human,lb_human,ub_human,_,_ = bootstrap_means(AH['obs_mean'])
axis.axhspan(lb_human,ub_human,
              color="black",
               alpha=0.1)

#plot means
## human
lb_human = abs(mean_human - lb_human) #as we error relative to mean
ub_human = abs(mean_human - ub_human)
axis.errorbar(0-.2,mean_human,
              yerr=([lb_human],[ub_human]),
              label="Human",
              color="black",
              marker = "o",
              ms=MARKER_SIZE*MEAN_MULTIPLIER)
## models
for x,(model,model_name) in enumerate(MODELS_trainings.items()):
    means = []
    lbs = []
    ubs = []
    for o,training in enumerate(TRAININGS):
        mean,lb,ub,_,_ = bootstrap_means(MH_trainings[(MH_trainings['Model Kind'] == model)&
                                                     (MH_trainings['Dynamics Training Dataset Type'] == training)]['model_correct'])
        lb = abs(mean-lb); ub = abs(mean-ub); 
        means.append(mean); lbs.append(lb), ubs.append(ub)
        axis.errorbar(x+.66+0.33 * o,mean,([lb],[ub]),
          marker="o",
          fillstyle=TRAINING_FILL[training],
          color=COLORS[model_name],
          label=model_name,
          ms=10)
    plt.plot([x+.66+0.33 * o for o in range(len(means))],means,color=COLORS[model_name])

#add labels
axis.set_xticks(np.arange(len(MODELS_trainings)+1))
axis.set_xticklabels(["Human"] + list(MODELS_trainings.values()),rotation=45,ha='right')
axis.set_title("Accuracy per dynamics training regimen")
axis.set_ylabel("Accuracy")
axis.set_ybound((0.3,1))
plt.savefig(os.path.join(plot_dir,"human_model_training_accuracy.pdf"),bbox_inches='tight')
plt.show()

## Scatter human/model over scenario
since DPI seems to be the most human-like (or maybe we could do this for a few of the models), we could look more finely at which scenarios it's making human or non-human errors on. So this could be a scatter plot with human performance on y, model performance on x, with one "dot" per scenario (i.e. using the colored icons.) Deviation from the line y = x is easily visible and indicates that the models still arent' making human-like predictions (+ whether that's because they're more or less accurate.)



In [ ]:
model = "DPI"

In [ ]:
MH_scatter = MH_both[MH_both['Model'] == model]

In [ ]:
fig, axis = plt.subplots(figsize=FIGSIZE)

#discrinimation line
axis.plot([0.5,1.],[0.5,1.],color="lightgrey")

for scenario in MH_scatter['scenario'].unique():
    row = MH_scatter[MH_scatter['scenario'] == scenario]
    axis.plot(row['model_correct'],
                row['human_correct'],
                marker = ICONS[scenario],
                color="black",
                markerfacecolor="lightgrey",
                ms=30)
    
axis.set_xbound((0.5,1))
axis.set_ybound((0.5,1))
axis.set_xlabel(model+" accuracy")
axis.set_ylabel("Human accuracy")
axis.set_title("Accuracy")
plt.savefig(os.path.join(plot_dir,"human_{}_accuracy_comp.pdf".format(model)),bbox_inches='tight')
plt.show()

## Scatter human/model over individual datapoints

### load human data

In [ ]:
from experiment_meta import *
HEM = pd.DataFrame(NEURIPS2021_EXPS) # HEM = "human experiment metadata"
HEM

In [ ]:
SCENARIOS = sorted([n.split("_")[0] for n in HEM['study'].unique()])

In [ ]:
## get paths to all human response data
data_paths = [os.path.join(csv_dir,'humans',i) for i in os.listdir(os.path.join(csv_dir,'humans'))]
resp_paths = [i for i in data_paths if i.split('/')[-1].split('-')[0]=='human_responses']
assert len(resp_paths)==8

In [ ]:
## also load all human data into a big dataframe
HD = pd.concat([h.apply_exclusion_criteria(h.load_and_preprocess_data(p)) for p in resp_paths])
print("Loaded {} lines".format(len(HD)))

#### exclude subjects from familiarization
Run `familiariarization_exclusion.ipynb` to generate `excluded_games.csv`

In [ ]:
bad_games = pd.read_csv(os.path.join(csv_dir,"humans/excluded_games.csv")).values[:,1]

In [ ]:
print("Excluding {} rows for {} games".format(sum(HD['gameID'].isin(bad_games)), len(bad_games)))
HD = HD[~HD['gameID'].isin(bad_games)]

### load model data

In [ ]:
## get paths to all model data
model_data_paths = [os.path.join(csv_dir,'models',i) for i in os.listdir(os.path.join(csv_dir,'models'))]
model_res_paths = [i for i in model_data_paths if i.split('.')[-1] == "csv"]

In [ ]:
## load all model results into a single dataframe
MD = pd.concat([pd.read_csv(p).assign(filename=p.split('/')[-1]) for p in model_res_paths])
print("Loaded {} rows".format(len(MD)))

In [ ]:
# a couple of import steps (restore original scenario names, add single prediction value, add correctness column)
MD = h.process_model_dataframe(MD)

In [ ]:
#check for duplicated rows
if len(MD.duplicated()) > 0:
    print("⚠️There are {} duplicated rows!".format(np.sum(MD.duplicated())))
    MD = MD[~MD.duplicated(h.MODEL_COLS+["Stimulus Name"],keep="first")]
    print("Removed duplicates, {} rows left".format(len(MD)))

In [ ]:
# save model kinds to variable
MODELS = list(MD["Model Kind"].unique())

#### exclude bad stims (where model/human stims mismatched)

In [ ]:
stim_comparision = pd.merge(pd.DataFrame(MD.groupby('Canon Stimulus Name')['Actual Outcome'].first()).reset_index(),pd.DataFrame(HD.groupby('stim_ID')['target_hit_zone_label'].first()).reset_index(),left_on='Canon Stimulus Name',right_on='stim_ID')

bad_stims = stim_comparision[stim_comparision['Actual Outcome'] != stim_comparision['target_hit_zone_label']]['Canon Stimulus Name']
print("There are {} bad stims".format(len(bad_stims)))

In [ ]:
#Exclude bad stims
HD = HD[~HD['stim_ID'].isin(bad_stims)]
MD = MD[~MD['Canon Stimulus Name'].isin(bad_stims)]

In [ ]:
#Also exclude stims from the rollingsliding ledge subset
HD = HD[~HD['stim_ID'].str.contains("rollingSliding_simple_ledge")]
MD = MD[~MD['Canon Stimulus Name'].str.contains("rollingSliding_simple_ledge")]

### The plot

In [ ]:
MD['correct']

In [ ]:
MD_both = MD[(MD['Readout Type'] == "B") & (MD['Dynamics Training Dataset Type'] == "all")]

In [ ]:

MODELS

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=4,figsize=(20,10),sharex=True,sharey=True)

for i,scenario in enumerate(SCENARIOS):
    axis = axes[int(i/4),i % 4]
    axis.plot([0.,1.],[0.,1.],color='lightgrey')
    human_responses = HD[HD['scenarioName']==scenario].groupby('stim_ID')['correct'].mean()
    stims = human_responses.index
    model_responses = MD_both[(MD_both['Canon Stimulus Name'].isin(stims)) & (MD_both['Model Kind'] == "DPI_nan_nan_nan_nan_L2 on particle 3D positions_1_same")].groupby('Canon Stimulus Name')['correct'].mean()
    model_responses.sort_index(inplace=True)
    human_responses.sort_index(inplace=True)
    assert np.all(model_responses.index == human_responses.index)
    #plot
    axis.scatter(model_responses,
                human_responses,
                color="black",s=10)
    axis.set_title(" ")  
    axis.set_xlabel(model+" accuracy")
    axis.set_ylabel("Human accuracy")
plt.savefig(os.path.join(plot_dir,"human_{}_accuracy_per_stim.pdf".format(model)),bbox_inches='tight')
plt.show()

In [ ]:
#get the most divergent stims
scenario = "dominoes"
human_responses = HD[HD['scenarioName']==scenario].groupby('stim_ID')['responseBool'].mean()
stims = human_responses.index
model_responses = MD[MD['Canon Stimulus Name'].isin(stims)].groupby('Canon Stimulus Name')['Predicted Outcome'].mean()
pd.DataFrame(abs(human_responses-model_responses)).sort_values(0).tail(5)